# SIS Model with No Demographics

Moving on from the SI model to something slightly more complex, we will add a state transition from infected back to susceptible, with no period of immunity - this is the SIS model.  It turns out that the equations governing this will be analogous to the SI model with demography - rather than births & mortality providing routes to add susceptibles and subtract infectives, this transition from infective -> susceptible will provide essentially the same mechanism. 

$$
\dot{S} = -\frac{\beta*S*I}{N} + \gamma I\\

\dot{I} = \frac{\beta*S*I}{N} - \gamma I
$$

As before, subbing $S = N-I$ into the second equation gives us

$$ \dot{I} = \beta I ( 1-\frac{\gamma}{\beta}-\frac{I}{N})$$

And comparing against the SI model with births, it is clear this equation is of the same form, with solution
$$
\frac{Nx}{1+(\frac{Nx}{I_0}-1)e^{-\beta x t}} \\
x = (1-\frac{\gamma}{\beta})
$$


This notebook tests the implementation and behavior of the model as follows:
### Construct the model
In the first few cells, we do all the necessary imports.  Then we construct a single-patch LASER model with three components: `Susceptibility`, `Transmission`, and `Infection_SIS` - this component will require a new agent property `itimer`, and upon expiration of `itimer` agents will return to the susceptible state.  Finally, we initialize with a single infection and run.  The `Susceptibility` and `Transmission` components are previously described. 


### Sanity check
The first test ensures certain basic constraints are being obeyed by the model.  We confirm that at each timestep, *S<sub>t</sub>=N<sub>t</sub>-I<sub>t</sub>*. 

### Scientific test
Finally, we come to the scientific test.  As before, we first test on a single instance of the model and show that the expected output is recovered.  Then, we select a few values of β and γ, run the model, fit the outputs to the logistic equation, and compare the fitted value of β and γ to the known values; all of the considerations noted in the SI with births model, in terms of how to approach this fit, are echoed again here.  Of particular concern is the approximation of an exponential transition from infected back to susceptible - as we are doing a first-order finite timestep integration here, that approximation will probably produce an error linear in γΔ*t* between the analytic result and the modeled result.  In fact, in a lot of real disease models, we have compartment dwell times in the exposed and infective states that are only a handful of Δ*t* long, but when doing real epi modeling and calibrating model parameters to uncertain data, this is generally not likely to be a dominant source of bias, uncertainty, etc.  But when comparing specifically against an analytic result, it can become significant.  



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from laser_core.propertyset import PropertySet
from scipy.optimize import curve_fit
from scipy.optimize import minimize

from laser_generic import Model
from laser_generic.infection import Infection_SIS
from laser_generic.susceptibility import Susceptibility
from laser_generic.transmission import Transmission

f"{np.__version__=}"

To make sure we don't accumulate lots of finite time-step error, make inf mean quite long in units of timestep

In [ ]:
pop = 3e5

scenario = pd.DataFrame(data=[["homenode", pop]], columns=["name", "population"])
parameters = PropertySet({"seed": 4, "nticks": 3000, "verbose": True, "beta": 0.01, "inf_mean": 300})
model = Model(scenario, parameters)
model.components = [
    Infection_SIS,
    Susceptibility,
    Transmission,
]

In [ ]:
from laser_generic.utils import seed_infections_randomly

seed_infections_randomly(model, ninfections=1)
model.run()

# Sanity checks
Check that the relationships between susceptible, infected, and total population hold.

In [ ]:
# I = model.patches.cases.astype("int")
# I2 = model.patches.populations[:-1] - model.patches.susceptibility.astype("int")
# plt.plot(I - I2)
plt.plot(model.patches.cases.astype("int"), lw=4)
plt.plot(model.patches.populations[:-1] - model.patches.susceptibility.astype("int"), "--", lw=3)
plt.yscale("log")
plt.legend(["Population minus currently infected", "Susceptible", "Population minus cumulative infections (incidence)"])
print("S = N-I:  " + str(np.isclose(model.patches.susceptibility, model.patches.populations[:-1] - model.patches.cases).all()))

### Single-simulation check
As before, starting with a single infection induces some stochasticity in terms of when the outbreak really starts to take off, and so we fit the expected behavior with a free offset parameter below.

In [ ]:
def SIS_logistic(t, beta, popsize, gamma, t0):
    x = 1 - gamma / beta
    return popsize * x / (1 + (popsize * x - 1) * np.exp(-beta * x * (t - t0)))


t = np.arange(model.params.nticks)


def objective(t0):
    return np.sum(
        (1 - SIS_logistic(t, model.params.beta, pop, 1 / model.params.inf_mean, t0) / np.squeeze(model.patches.cases.astype("int"))) ** 2
    )


result = minimize(objective, x0=10)
t0_opt = result.x[0]

plt.plot(model.patches.cases, lw=4)
plt.plot(SIS_logistic(t, model.params.beta, pop, 1 / model.params.inf_mean, 0), lw=3)
plt.plot(SIS_logistic(t, model.params.beta, pop, 1 / model.params.inf_mean, t0_opt), "r:", lw=3)
plt.yscale("log")
plt.legend(["Model output", "Logistic growth with known inputs, t0=0", f"Logistic growth with known inputs, best-fit t0 = {t0_opt:.1f}"])

## Scientific testing
Finally, we run the model for a range of β & γ  parameters, we freely fit the model output to the logistic equation, and we compare the known input parameters against the parameters fitted from output.  

We will use only relatively large values of γ for this procedure.  The reason why will become clear in a second test, where we demonstrate that there is an error between the expected final size and the modeled final size, and that this error shrinks with γΔ*t*, as we would expect a first-order approximation error to.  

To make this a pass-fail test, we will raise a flag if the fitted parameters are more than 5% different than the known ones.

In [ ]:
%%capture

nticks = 3000
t = np.arange(nticks)
betarange = [0.03, 0.1]
gammarange = [1 / 200, 1 / 50]
seeds = list(range(10))
pop = 3e5
betas = np.random.uniform(betarange[0], betarange[1], 10)
gammas = np.random.uniform(gammarange[0], gammarange[1], 10)
output = pd.DataFrame(columns=["seed", "beta", "gamma", "cases", "fitted_beta", "fitted_gamma", "fitted_t0"])
scenario = pd.DataFrame(data=[["homenode", pop]], columns=["name", "population"])

for seed, beta, gamma in zip(seeds, betas, gammas):
    parameters = PropertySet({"seed": seed, "nticks": nticks, "verbose": True, "beta": beta, "inf_mean": 1 / gamma})
    model = Model(scenario, parameters)
    model.components = [
        Infection_SIS,
        Susceptibility,
        Transmission,
    ]
    seed_infections_randomly(model, ninfections=3)
    model.run()
    cases = [model.patches.cases[i][0] for i in range(nticks)]
    popt, pcov = curve_fit(
        SIS_logistic,
        t,
        cases,
        p0=[np.mean(betarange), pop, np.mean(gammarange), 1],
        bounds=([betarange[0] / 2, pop - 1, gammarange[0] / 2, -300], [betarange[1] * 2, pop + 1, gammarange[1] * 2, 300]),
    )

    output = pd.concat(
        [
            output,
            pd.DataFrame.from_dict(
                {
                    "seed": seed,
                    "beta": beta,
                    "gamma": gamma,
                    "cases": [np.array(cases)],
                    "fitted_beta": popt[0],
                    "fitted_gamma": popt[2],
                    "fitted_t0": popt[3],
                }
            ),
        ],
        ignore_index=True,
    )

In [ ]:
plt.figure()
plt.plot(output["beta"], output["fitted_beta"], "o")
plt.xlim(betarange[0], betarange[1])
plt.ylim(betarange[0], betarange[1])
plt.figure()
plt.plot(output["beta"], output["beta"] - output["fitted_beta"], "o")
plt.xlim(betarange[0], betarange[1])
plt.ylim(-betarange[1], betarange[1])
plt.figure()
plt.plot(output["gamma"], output["fitted_gamma"], "o")
plt.xlim(gammarange[0], gammarange[1])
plt.ylim(gammarange[0], gammarange[1])
plt.figure()
plt.plot(output["gamma"], output["gamma"] - output["fitted_gamma"], "o")
plt.xlim(gammarange[0], gammarange[1])
plt.ylim(-gammarange[1], gammarange[1])

In [ ]:
print(
    "All fitted beta are within 5% of known beta: " + str(np.all(np.abs((output["beta"] - output["fitted_beta"]) / output["beta"]) < 0.05))
)
print(
    "All fitted gamma are within 10% of known gamma: "
    + str(np.all(np.abs((output["gamma"] - output["fitted_gamma"]) / output["gamma"]) < 0.1))
)

In [ ]:
output

### Quick demonstration of first-order error accumulation
As noted before, for γΔ*t* large, first-order integration like we are doing here can accumulate substantial error.  Calculating exactly how error will accumulate in an integrator for a dynamic process like this is beyond the scope here, and probably depends on a lot of details.  E.g., the ordering of steps - in a given step, does the `transmission` update from S->I state occur before or after the `infection` update that sends agents from I->S?  Do we use midpoint methods, timer countdowns, or take advantage of the unique memorylessness of the exponential distribution to simply remove a random fraction each time?  All that is beyond scope here, but just want to demonstrate that the error in the equilibrium value *I(t -> ∞)* becomes large when the mean infectious period *1/γ* is on the same order as Δ*t*.

In [ ]:
%%capture

gammas = [1 / infmean for infmean in [1, 1.5, 2, 2.5, 3, 6, 12, 18, 30, 45, 60, 90, 120, 180, 240, 300]]
betas = [3 * gamma for gamma in gammas]


nticks = 3000
seeds = list(range(len(gammas)))
pop = 1e5
final_expected = np.array([])
final_observed = np.array([])
scenario = pd.DataFrame(data=[["homenode", pop]], columns=["name", "population"])

for seed, beta, gamma in zip(seeds, betas, gammas):
    parameters = PropertySet({"seed": seed, "nticks": nticks, "verbose": True, "beta": beta, "inf_mean": 1 / gamma})
    model = Model(scenario, parameters)
    model.components = [
        Infection_SIS,
        Susceptibility,
        Transmission,
    ]
    seed_infections_randomly(model, ninfections=20)
    model.run()
    final_observed = np.append(final_observed, model.patches.cases[-1][0])
    final_expected = np.append(final_expected, pop * (1 - gamma / beta))

In [ ]:
plt.plot(gammas, np.abs(1 - final_observed / final_expected), "o")
plt.xlabel(r"$\gamma$")
plt.ylabel("$| 1 - \\frac{I(\\infty)_{obs}}{I(\\infty)_{exp}} |$")
plt.title(r"Error in equilibrium infected fraction increases roughly linearly in $\gamma \Delta t$")